In [1]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SQLContext
from utilities import statistics, post_processing

In [2]:
tstat_train_path =  '/home/yolanda/workspace/tcp_temp_complete_tagged_ok'

sc = pyspark.SparkContext()
sqlCxt = SQLContext(sc)

In [3]:
df_train = sqlCxt.read.format('com.databricks.spark.csv').options(header='true', delimiter=" ", inferschema='true').load(tstat_train_path)

In [3]:
train_subsampled = post_processing.subsampling(sc, sqlCxt, df_train, ranges=None, bins=4, sample_ratios=[1, 1/10,1/20,1/30], take_first=None)

NameError: name 'df_train' is not defined

In [6]:
train_subsampled.count()

85680

In [4]:
tstat_test_path =  '/home/yolanda/workspace/tcp_temp_complete_tagged_ok'
df_test = sqlCxt.read.format('com.databricks.spark.csv').options(header='true', delimiter=" ", inferschema='true').load(tstat_test_path).limit(1000)

In [5]:
test_subsampled = post_processing.subsampling(sc, sqlCxt, df_test, ranges=[(0,5),(5,'n')], bins=None, sample_ratios=None, take_first=['n',5])

In [6]:
test_subsampled.count()

1000

In [31]:
flows_id.count()

0

In [11]:
df_ranked_flow_filtered.count()

0

In [17]:
print(flow_id.collect())

TypeError: 'Column' object is not callable

In [ ]:
test_subsampled.printSchema()

In [ ]:
test_subsampled.drop('tag:132') # drop tag for testing
n_classes = train_subsampled.select('tag:132').distinct().count()

In [ ]:
from pyspark.mllib.tree import RandomForest, RandomForestModel
from pyspark.mllib.util import MLUtils

# Load and parse the data file into an RDD of LabeledPoint.
data = MLUtils.loadLibSVMFile(sc, 'data/mllib/sample_libsvm_data.txt')
# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = data.randomSplit([0.7, 0.3])

# Train a RandomForest model.
#  Empty categoricalFeaturesInfo indicates all features are continuous.
#  Note: Use larger numTrees in practice.
#  Setting featureSubsetStrategy="auto" lets the algorithm choose.
model = RandomForest.trainClassifier(train_subsampled, numClasses=n_classes, categoricalFeaturesInfo={},
                                     numTrees=3, featureSubsetStrategy="auto",
                                     impurity='gini', maxDepth=4, maxBins=32)

# Evaluate model on test instances and compute test error
predictions = model.predict(test_subsampled.map(lambda x: x.features))
labelsAndPredictions = test_subsampled.map(lambda lp: lp.label).zip(predictions)
testErr = labelsAndPredictions.filter(
    lambda lp: lp[0] != lp[1]).count() / float(test_subsampled.count())
print('Test Error = ' + str(testErr))
print('Learned classification forest model:')
print(model.toDebugString())

# Save and load model
model.save(sc, "target/tmp/myRandomForestClassificationModel")
sameModel = RandomForestModel.load(sc, "target/tmp/myRandomForestClassificationModel")